# 🧪 Teste Completo do Pipeline com Visualizações

Este notebook testa todo o pipeline e gera visualizações dos dados.

## 1. Executar Teste Completo

Execute a célula abaixo para:
- Verificar dados Bronze
- Executar ingestão se necessário
- Verificar dados Prata
- Executar transformação se necessário
- Gerar gráficos e análises

In [ ]:
# Executar teste completo
exec(open('/home/jovyan/work/teste_completo_com_graficos.py').read())

## 2. Visualizar Gráficos Gerados

Se os gráficos foram salvos, você pode visualizá-los aqui:

In [ ]:
from IPython.display import Image, display
import os

dashboard_path = '/home/jovyan/work/dashboard_analise.png'
if os.path.exists(dashboard_path):
    display(Image(dashboard_path))
else:
    print("⚠️  Dashboard ainda não foi gerado. Execute a célula acima primeiro.")

## 3. Análise Interativa dos Dados

Explore os dados carregados:

In [ ]:
# Carregar dados Prata
from pyspark.sql import SparkSession

# Garantir que Spark está inicializado
try:
    spark
except NameError:
    exec(open('/home/jovyan/work/spark_com_jars_manual.py').read())
    spark = globals()['spark']

# Carregar dados
df_fato_bf = spark.read.parquet("s3a://govbr/prata/fato_bolsa_familia/")
df_estados = spark.read.parquet("s3a://govbr/prata/dim_estados/")

# Converter para Pandas
df_bf_pd = df_fato_bf.toPandas()
df_estados_pd = df_estados.toPandas()

print("✅ Dados carregados!")
print(f"\nFato Bolsa Família: {len(df_bf_pd)} registros")
print(f"Estados: {len(df_estados_pd)} registros")

In [ ]:
# Ver primeiras linhas do Fato Bolsa Família
if 'df_bf_pd' in locals() and len(df_bf_pd) > 0:
    print("\n📊 Primeiras linhas do Fato Bolsa Família:")
    display(df_bf_pd.head(10))
    
    print("\n📈 Colunas disponíveis:")
    print(df_bf_pd.columns.tolist())
else:
    print("⚠️  Dados não disponíveis")

In [ ]:
# Estatísticas descritivas
if 'df_bf_pd' in locals() and len(df_bf_pd) > 0:
    print("\n📊 Estatísticas Descritivas - Bolsa Família:")
    
    if 'quantidade_beneficiarios' in df_bf_pd.columns:
        print(f"\nTotal de Beneficiários: {df_bf_pd['quantidade_beneficiarios'].sum():,.0f}")
        print(f"Média por Município: {df_bf_pd['quantidade_beneficiarios'].mean():,.0f}")
        print(f"Mediana: {df_bf_pd['quantidade_beneficiarios'].median():,.0f}")
        print(f"Mínimo: {df_bf_pd['quantidade_beneficiarios'].min():,.0f}")
        print(f"Máximo: {df_bf_pd['quantidade_beneficiarios'].max():,.0f}")
    
    if 'valor_total' in df_bf_pd.columns:
        print(f"\nValor Total: R$ {df_bf_pd['valor_total'].sum()/1e6:.2f} milhões")
        print(f"Média por Município: R$ {df_bf_pd['valor_total'].mean()/1e3:.2f} mil")
    
    if 'percentual_beneficiarios' in df_bf_pd.columns:
        print(f"\n% Médio de População Assistida: {df_bf_pd['percentual_beneficiarios'].mean():.2f}%")
        print(f"% Máximo: {df_bf_pd['percentual_beneficiarios'].max():.2f}%")
        print(f"% Mínimo: {df_bf_pd['percentual_beneficiarios'].min():.2f}%")
else:
    print("⚠️  Dados não disponíveis")

In [ ]:
# Criar gráficos interativos adicionais
import matplotlib.pyplot as plt
import seaborn as sns

if 'df_bf_pd' in locals() and len(df_bf_pd) > 0:
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    
    # 1. Top 10 Municípios por Beneficiários
    if 'quantidade_beneficiarios' in df_bf_pd.columns:
        municipio_col = 'nome_municipio' if 'nome_municipio' in df_bf_pd.columns else 'municipio'
        if municipio_col in df_bf_pd.columns:
            top10 = df_bf_pd.nlargest(10, 'quantidade_beneficiarios')
            axes[0, 0].barh(top10[municipio_col], top10['quantidade_beneficiarios'] / 1000, color='steelblue')
            axes[0, 0].set_xlabel('Beneficiários (mil)')
            axes[0, 0].set_title('Top 10 Municípios - Beneficiários')
            axes[0, 0].invert_yaxis()
    
    # 2. Top 10 Municípios por % Assistido
    if 'percentual_beneficiarios' in df_bf_pd.columns:
        municipio_col = 'nome_municipio' if 'nome_municipio' in df_bf_pd.columns else 'municipio'
        if municipio_col in df_bf_pd.columns:
            top10_perc = df_bf_pd.nlargest(10, 'percentual_beneficiarios')
            axes[0, 1].barh(top10_perc[municipio_col], top10_perc['percentual_beneficiarios'], color='coral')
            axes[0, 1].set_xlabel('% População Assistida')
            axes[0, 1].set_title('Top 10 Municípios - % Assistido')
            axes[0, 1].invert_yaxis()
    
    # 3. Distribuição de Beneficiários
    if 'quantidade_beneficiarios' in df_bf_pd.columns:
        axes[1, 0].hist(df_bf_pd['quantidade_beneficiarios'] / 1000, bins=20, color='skyblue', edgecolor='black')
        axes[1, 0].set_xlabel('Beneficiários (mil)')
        axes[1, 0].set_ylabel('Frequência')
        axes[1, 0].set_title('Distribuição de Beneficiários')
        axes[1, 0].grid(True, alpha=0.3)
    
    # 4. Beneficiários vs População
    if 'quantidade_beneficiarios' in df_bf_pd.columns and 'populacao' in df_bf_pd.columns:
        axes[1, 1].scatter(df_bf_pd['populacao'] / 1000, df_bf_pd['quantidade_beneficiarios'] / 1000, 
                          alpha=0.6, s=50, color='purple')
        axes[1, 1].set_xlabel('População (mil)')
        axes[1, 1].set_ylabel('Beneficiários (mil)')
        axes[1, 1].set_title('Beneficiários vs População')
        axes[1, 1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
else:
    print("⚠️  Dados não disponíveis para visualização")

## 4. Consultas SQL (Opcional)

Execute consultas SQL nos dados:

In [ ]:
# Criar views temporárias
if 'df_fato_bf' in locals():
    df_fato_bf.createOrReplaceTempView("fato_bolsa_familia")
    
if 'df_estados' in locals():
    df_estados.createOrReplaceTempView("dim_estados")

# Exemplo: Top 10 municípios por beneficiários
if 'df_fato_bf' in locals():
    resultado = spark.sql("""
        SELECT 
            nome_municipio,
            uf_sigla,
            quantidade_beneficiarios,
            valor_total,
            percentual_beneficiarios
        FROM fato_bolsa_familia
        ORDER BY quantidade_beneficiarios DESC
        LIMIT 10
    """)
    
    print("\n🏆 Top 10 Municípios por Beneficiários:")
    resultado.show(truncate=False)